<a href="https://colab.research.google.com/github/digs1998/Fine-Tuning-LLMs-LoRA/blob/main/Fine_tuning_LLM_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch \
    torchdata --quiet

%pip install \
    transformers \
    datasets \
    evaluate \
    loralib \
    peft \
    huggingface_hub \
    accelerate \
    trl --quiet

%pip install -U bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 129.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 M

In [2]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

## Use Pretrained BERT for Sentiment Analysis

In [3]:
model_name = 'distilbert-base-uncased' #67 Million parameters

# Label maps for Sentiment Analysis
id2label = {0: "Negative", 1 : "Positive"}
label2id = {"Negative":0, "Positive": 1}

## generate classification model from checkpoint
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2,
                                                           id2label = id2label, label2id=label2id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
dataset = load_dataset("shawhin/imdb-truncated") #movie reviews
dataset

README.md:   0%|          | 0.00/592 [00:00<?, ?B/s]

(…)-00000-of-00001-5a744bf76a1d84b2.parquet:   0%|          | 0.00/836k [00:00<?, ?B/s]

(…)-00000-of-00001-a3a52fabb70c739f.parquet:   0%|          | 0.00/853k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

### Preprocess Dataset

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space = True)

def tokenize_data(example):
  ## extract text
  text = example['text']

  ## tokenize, and truncate data, making sure the lenght remains same
  tokenizer.truncation_side = "left"
  tokenized_inputs = tokenizer(
      text,
      return_tensors = "np",
      truncation = True,
      max_length = 512
  )

  return tokenized_inputs

# if pad token does not exist, add pad token
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': "[PAD]"})
  model.resize_token_embeddings(len(tokenizer))

tokenized_dataset = dataset.map(tokenize_data, batched=True)
tokenized_dataset

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [6]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer) #dynamically pads examples in the batch, to be as long as the longest sequence in the batch

acc = evaluate.load("accuracy")

In [7]:
def compute_metrics(p):
  preds, labels = p
  preds = np.argmax(preds, axis=1)

  return {"accuracy": acc.compute(predictions = preds, references=labels)}

## Zero Shot Inferencing

In [8]:
text_list = ["It was great, absolutely loved it", "It was alright, but could've been better",
             "A huge disappointment", "Not worth a watch", "I liked the first one better"]

print("---- Zero Shot Inferencing -----")

for text in text_list:
  # tokenize the text
  inputs = tokenizer.encode(text, return_tensors="pt")
  # compute logits
  logits = model(inputs).logits
  # convert logits to labels
  preds = torch.argmax(logits)

  print(text + " - " + id2label[preds.tolist()])

---- Zero Shot Inferencing -----
It was great, absolutely loved it - Negative
It was alright, but could've been better - Negative
A huge disappointment - Negative
Not worth a watch - Negative
I liked the first one better - Negative


## LoRA Implementation

In [9]:
peft_config = LoraConfig(task_type = "SEQ_CLS", #sequence classification
                         r=4, #intrinsic rank of trainable weight matrix
                         lora_alpha = 16, # learning rate
                         target_modules = ['q_lin'] # applying PEFT to query layer
                         )

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [10]:
training_args = TrainingArguments(
    output_dir = model_name + "-lora-text-classification",
    learning_rate = 1e-4,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    num_train_epochs = 15,
    weight_decay = 0.001,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["validation"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-89015f90810e>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: digvijayyadav48 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.295067,{'accuracy': 0.873}
2,0.444200,0.348252,{'accuracy': 0.873}
3,0.444200,0.325612,{'accuracy': 0.89}
4,0.299800,0.374992,{'accuracy': 0.89}
5,0.299800,0.371525,{'accuracy': 0.896}
6,0.259000,0.372853,{'accuracy': 0.899}
7,0.259000,0.386847,{'accuracy': 0.899}
8,0.232000,0.398933,{'accuracy': 0.901}
9,0.232000,0.418426,{'accuracy': 0.9}
10,0.218500,0.419867,{'accuracy': 0.904}


Trainer is attempting to log a value of "{'accuracy': 0.873}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.873}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.89}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.89}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.896}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This inv

TrainOutput(global_step=3750, training_loss=0.2564993072509766, metrics={'train_runtime': 737.9547, 'train_samples_per_second': 20.326, 'train_steps_per_second': 5.082, 'total_flos': 1672490471327616.0, 'train_loss': 0.2564993072509766, 'epoch': 15.0})

## Test the Model Performance

In [12]:
model.to("cuda:0")

print("---- Inferencing ----")
for text in text_list:
  # tokenize the text
  inputs = tokenizer.encode(text, return_tensors="pt").to("cuda:0")
  # compute logits
  logits = model(inputs).logits
  # convert logits to labels
  preds = torch.argmax(logits)

  print(text + " - " + id2label[preds.tolist()])

---- Inferencing ----
It was great, absolutely loved it - Positive
It was alright, but could've been better - Negative
A huge disappointment - Negative
Not worth a watch - Negative
I liked the first one better - Negative
